# Ham or Spam?

In [1]:
# when installing nltk for the first time we need to also download a few built in libraries
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd

#df = pd.read_csv("emails_2.csv")


df = pd.read_pickle("email")
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


The dataset is made up of email that are classified as ham [0] or spam[1]. You need to clean the dataset before training a prediction model.

## Remove Punctuation

👇 Create a function to remove the punctuation. Apply it to the entire data and add the output as a new column in the dataframe called `clean_text`

In [3]:
import string

def  remove_punctuation(new_colum, old_colum):
    df[new_colum] = df[old_colum].apply(lambda x: ''.join([c for c in x if c not in string.punctuation]))
    return df

remove_punctuation("clean_text","text")
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...


## Lower Case

👇 Create a function to lower case the text. Apply it to `clean_text`

In [4]:
def lower_case(colum_name):
    df[colum_name]= df[colum_name].str.lower()
    return df


In [5]:
lower_case("clean_text")
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",1,subject do not have money get software cds fr...


## Remove Numbers

👇 Create a function to remove numbers from the text. Apply it to `clean_text`

In [6]:
def remove_numbers(text):
    return ''.join([c for c in text if not c.isdigit()])

df = df.assign(clean_text = df["clean_text"].apply(remove_numbers))
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,subject color printing special request addit...
4,"Subject: do not have money , get software cds ...",1,subject do not have money get software cds fr...


## Remove StopWords

👇 Create a function to remove stopwords from the text. Apply it to `clean_text`.

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stopwords(language, column):
    stop_words = set(stopwords.words(language))
    return df[column].apply(lambda x: [word for word in x.split() if word not in stop_words])

df = df.assign(clean_text = remove_stopwords("english", "clean_text"))
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,"[subject, naturally, irresistible, corporate, ..."
1,Subject: the stock trading gunslinger fanny i...,1,"[subject, stock, trading, gunslinger, fanny, m..."
2,Subject: unbelievable new homes made easy im ...,1,"[subject, unbelievable, new, homes, made, easy..."
3,Subject: 4 color printing special request add...,1,"[subject, color, printing, special, request, a..."
4,"Subject: do not have money , get software cds ...",1,"[subject, money, get, software, cds, software,..."


## Lemmatize

👇 Create a function to lemmatize the text. Make sure the output is a single string, not a list of words. Apply it to `clean_text`.

In [12]:

from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('omw-1.4')
def lemmatize(text):
    lmtzr = WordNetLemmatizer()
    #df['clean_text'] = df['clean_text'].apply(lambda lst:[lmtzr.lemmatize(word) for word in lst])
    a = [lmtzr.lemmatize(word) for word in text ]
    #df['clean_text'] = " ".join(df["clean_text"])
    return " ".join(a)

In [20]:
df["clean_text"] = df["clean_text"].apply(lemmatize)

In [21]:
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,1,subject stock trading gunslinger fanny merrill...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new home made easy im wan...
3,Subject: 4 color printing special request add...,1,subject color printing special request additio...
4,"Subject: do not have money , get software cds ...",1,subject money get software cd software compati...


## Bag-of-words Modelling

👇 Vectorize the `clean_text` to a Bag-of-Words representation with a default CountVectorizer . Save as `X_bow`.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(df.clean_text)

#X_bow.toarray()

In [ ]:
pd.DataFrame(X_bow.toarray(), columns = vectorizer.get_feature_names())

👇 Cross-validate a MultinomialNB model with the Bag-of-words. Score the model's accuracy.

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

mnb = MultinomialNB()
scores = cross_val_score(mnb, X_bow, df['spam'], cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.99 (+/- 0.00)


⚠️ Please push the exercise once you are done 🙃

## 🏁 